In [31]:
import sys
from pathlib import Path
from src.data_utils import load_raw_data
from datetime import datetime, timedelta

# Añade src al path para importar los módulos
sys.path.append(str(Path().resolve().parent / 'src'))

import config

In [32]:
%reload_ext autoreload
%autoreload 2

In [40]:
from datetime import datetime, timedelta
import pandas as pd

# Fecha actual redondeada al día
current_date = pd.to_datetime(datetime.utcnow()).floor('D')
print(f"Fecha actual redondeada: {current_date}")

# Encuentra el lunes de esta semana
monday_this_week = current_date - timedelta(days=current_date.weekday())
# Encuentra el lunes de la semana pasada
monday_last_week = monday_this_week - timedelta(days=7)
print(f"Lunes de la semana pasada: {monday_last_week}")

Fecha actual redondeada: 2025-08-10 00:00:00
Lunes de la semana pasada: 2025-07-28 00:00:00


In [46]:
def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    print(f"Obteniendo datos desde {from_date} hasta {to_date}")
    sales_week = load_raw_data(fecha_inicio=from_date, fecha_fin=to_date - timedelta(days=1))
    # Filtrar solo la semana si tiene los 7 días completos
    dias_unicos = sales_week['fecha'].nunique()
    if dias_unicos == 7:
        sales_week = sales_week[(sales_week['fecha'] >= from_date) & (sales_week['fecha'] < to_date)]
        sales_week.sort_values(by='fecha', inplace=True)
        return sales_week
    else:
        print('No hay una semana completa en el rango dado.')
        return pd.DataFrame()

In [47]:
# Usar las variables definidas previamente en el notebook para las fechas
from_date = monday_last_week
to_date = monday_this_week
df = fetch_batch_raw_data(from_date, to_date)
print(f"Datos obtenidos: {len(df)} filas")

Obteniendo datos desde 2025-07-28 00:00:00 hasta 2025-08-04 00:00:00
Cargando datos desde: C:\Workspace\mlops_fleca_project\data\raw\raw_data_bq_forecasting_20250810.parquet
Total de fechas faltantes: 1
Fechas faltantes: ['2025-08-03T00:00:00.000000000']
No hay una semana completa en el rango dado.
Datos obtenidos: 0 filas
